In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykalman import KalmanFilter

# --- Step 1: Generate synthetic data ---
np.random.seed(42)

n_timesteps = 100
true_trend_start = 50
true_trend_rate = 0.3
seasonal_amplitude = 10
seasonal_period = 20
noise_std = 1.5

# Generate trend
trend = true_trend_start + true_trend_rate * np.arange(n_timesteps)

# Generate seasonal component (sine wave)
seasonality = seasonal_amplitude * np.sin(2 * np.pi * np.arange(n_timesteps) / seasonal_period)

# Add noise
noise = np.random.normal(0, noise_std, size=n_timesteps)

# Final observed demand
observed_demand = trend + seasonality + noise

# --- Step 2: Kalman filter to estimate linear trend ---
kf = KalmanFilter(
    transition_matrices=[[1, 1], [0, 1]],
    observation_matrices=[[1, 0]],
    transition_covariance=np.eye(2) * 0.1  ,
    observation_covariance=noise_std**2,
    initial_state_mean=[observed_demand[0], 0]
)

state_means, _ = kf.smooth(observed_demand)
estimated_trend = state_means[:, 0]

# --- Step 3: Calculate residuals (historical seasonal signal) ---
seasonal_residuals = observed_demand - estimated_trend

# --- Step 4: Reconstruct using trend + raw residuals (data-driven) ---
reconstructed_demand = estimated_trend + seasonal_residuals

# --- Step 5: Plot ---
plt.figure(figsize=(12, 7))

plt.plot(observed_demand, 'k.', label='Observed Demand (Noisy)', alpha=0.5)
plt.plot(trend, 'g--', label='True Trend')
plt.plot(estimated_trend, 'b-', label='Estimated Trend (Kalman)')
plt.plot(reconstructed_demand, 'r--', label='Reconstructed Demand (Trend + Residuals)')

plt.title('Data-Driven Trend + Seasonality Reconstruction via Kalman Filter')
plt.xlabel('Time')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3427083353.py, line 31)